# Burgers Identification

In [1]:
from pathlib import Path
import sys

utils_path = Path('../utils').resolve()
sys.path.insert(0, str(utils_path))

In [2]:
import torch

from collections import OrderedDict

import numpy as np

import matplotlib.pyplot as plt

import scipy.io

from scipy.interpolate import griddata

from graphs_utils import newfig, savefig

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
print(f'Using {device}')

Using cuda


In [5]:
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        
        # parameters
        self.depth = len(layers) - 1
        
        # set up layer order dict
        self.activation = torch.nn.Tanh
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        layerDict = OrderedDict(layer_list)
        
        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [6]:
class PhysicsInformedNN():
    def __init__(self, X, u, layers, lb, ub):
        
        # boundary conditions
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)
        
        # data
        self.x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        self.t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)
        self.u = torch.tensor(u).float().to(device)
        
        # settings
        self.lambda_1 = torch.tensor([0.0], requires_grad=True).to(device)
        self.lambda_2 = torch.tensor([-6.0], requires_grad=True).to(device)
        
        self.lambda_1 = torch.nn.Parameter(self.lambda_1)
        self.lambda_2 = torch.nn.Parameter(self.lambda_2)
        
        # deep neural networks
        self.dnn = DNN(layers).to(device)
        self.dnn.register_parameter('lambda_1', self.lambda_1)
        self.dnn.register_parameter('lambda_2', self.lambda_2)
        
         # optimizers: using the same settings
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"       # can be "strong_wolfe"
        )
        
        self.optimizer_Adam = torch.optim.Adam(self.dnn.parameters())
        self.iter = 0
        
    def net_u(self, x, t):  
        u = self.dnn(torch.cat([x, t], dim=1))
        return u
    
    def net_f(self, x, t):
        """ The pytorch autograd version of calculating residual """
        lambda_1 = self.lambda_1        
        lambda_2 = torch.exp(self.lambda_2)
        u = self.net_u(x, t)
        
        u_t = torch.autograd.grad(
            u, t, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_x = torch.autograd.grad(
            u, x, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_xx = torch.autograd.grad(
            u_x, x, 
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True
        )[0]
        
        f = u_t + lambda_1 * u * u_x - lambda_2 * u_xx
        return f
    
    def loss_func(self):
        u_pred = self.net_u(self.x, self.t)
        f_pred = self.net_f(self.x, self.t)
        loss = torch.mean((self.u - u_pred) ** 2) + torch.mean(f_pred ** 2)
        self.optimizer.zero_grad()
        loss.backward()
        
        self.iter += 1
        if self.iter % 100 == 0:
            print(
                'Loss: %e, l1: %.5f, l2: %.5f' % 
                (
                    loss.item(), 
                    self.lambda_1.item(), 
                    torch.exp(self.lambda_2.detach()).item()
                )
            )
        return loss
    
    # def train(self, nIter):
    #     self.dnn.train()
    #     for epoch in range(nIter):
    #         u_pred = self.net_u(self.x, self.t)
    #         f_pred = self.net_f(self.x, self.t)
    #         loss = torch.mean((self.u - u_pred) ** 2) + torch.mean(f_pred ** 2)
            
    #         # Backward and optimize
    #         self.optimizer_Adam.zero_grad()
    #         loss.backward()
    #         self.optimizer_Adam.step()
            
    #         if epoch % 100 == 0:
    #             print(
    #                 'It: %d, Loss: %.3e, Lambda_1: %.3f, Lambda_2: %.6f' % 
    #                 (
    #                     epoch, 
    #                     loss.item(), 
    #                     self.lambda_1.item(), 
    #                     torch.exp(self.lambda_2).item()
    #                 )
    #             )
                
    #     # Backward and optimize
    #     self.optimizer.step(self.loss_func)
    
    def train(self, nIter):
        self.dnn.train()
        pbar = tqdm(range(nIter), desc="Training", ncols=100)
        for epoch in pbar:
            u_pred = self.net_u(self.x, self.t)
            f_pred = self.net_f(self.x, self.t)
            loss = torch.mean((self.u - u_pred) ** 2) + torch.mean(f_pred ** 2)
            
            # Backward and optimize
            self.optimizer_Adam.zero_grad()
            loss.backward()
            self.optimizer_Adam.step()

            # Update the progress bar with the loss information
            pbar.set_postfix(
                Loss=f"{loss.item():.3e}", 
                Lambda_1=f"{self.lambda_1.item():.3f}", 
                Lambda_2=f"{torch.exp(self.lambda_2).item():.6f}",
                refresh=True
            )
                
        # Backward and optimize
        self.optimizer.step(self.loss_func)
    
    
    def predict(self, X):
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)

        self.dnn.eval()
        u = self.net_u(x, t)
        f = self.net_f(x, t)
        u = u.detach().cpu().numpy()
        f = f.detach().cpu().numpy()
        return u, f

### Configurations

In [7]:
nu = 0.01/np.pi

N_u = 2000
layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]

data = scipy.io.loadmat('data/burgers_shock.mat')

t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.flatten()[:,None]              

# Doman bounds
lb = X_star.min(0)
ub = X_star.max(0) 

### Training

In [8]:
%%time

noise = 0.0

# create training set
idx = np.random.choice(X_star.shape[0], N_u, replace=False)
X_u_train = X_star[idx,:]
u_train = u_star[idx,:]

# training
model = PhysicsInformedNN(X_u_train, u_train, layers, lb, ub)
model.train(0)

Loss: 2.310215e-02, l1: 0.20404, l2: 0.00239
Loss: 1.550285e-02, l1: 0.44315, l2: 0.00253
Loss: 8.796914e-03, l1: 0.59975, l2: 0.00278
Loss: 7.027099e-03, l1: 0.66012, l2: 0.00279
Loss: 4.600182e-03, l1: 0.73827, l2: 0.00295
Loss: 2.721438e-03, l1: 0.81310, l2: 0.00317
Loss: 1.762194e-03, l1: 0.88132, l2: 0.00348
Loss: 1.198461e-03, l1: 0.92897, l2: 0.00368
Loss: 8.939077e-04, l1: 0.95892, l2: 0.00377
Loss: 7.051424e-04, l1: 0.96341, l2: 0.00385
Loss: 5.238830e-04, l1: 0.98313, l2: 0.00399
Loss: 3.948458e-04, l1: 0.97661, l2: 0.00396
Loss: 2.949358e-04, l1: 0.98505, l2: 0.00381
Loss: 2.119915e-04, l1: 0.99167, l2: 0.00367
Loss: 1.745338e-04, l1: 0.98633, l2: 0.00352
Loss: 1.398610e-04, l1: 0.98692, l2: 0.00344
Loss: 1.123266e-04, l1: 0.99513, l2: 0.00336
Loss: 9.552659e-05, l1: 0.99338, l2: 0.00328
Loss: 8.419155e-05, l1: 0.99151, l2: 0.00324
Loss: 7.499390e-05, l1: 0.99365, l2: 0.00323
Loss: 6.587169e-05, l1: 0.99272, l2: 0.00322
Loss: 6.012571e-05, l1: 0.99300, l2: 0.00321
Loss: 5.34